In [ ]:
# Makes the necessary imports
import sys
from datetime import datetime
import time
import os

from facebook_business import apiconfig
apiconfig.ads_api_config['API_VERSION'] = 'v14.0'
for mod in list(sys.modules):
    if mod.startswith('facebook_business') and mod != 'facebook_business.apiconfig':
        del sys.modules[mod]
        
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.adset import AdSet
from facebook_business.adobjects.adcreative import AdCreative
from telescope_base import pg_driver
import psycopg2
import psycopg2.errorcodes

In [ ]:
# Credentials
my_app_id = os.environ['FB_ADS_APP_ID']
my_app_secret = os.environ['FB_ADS_APP_SECRET']
my_access_token = os.environ['FB_ADS_ACCESS_TOKEN']
FacebookAdsApi.init(my_app_id, my_app_secret, my_access_token)
my_account = AdAccount('act_585359792073028')

In [ ]:
# Get campaign's ids
campaigns = my_account.get_campaigns()

In [ ]:
# Campaing's fields and params
campaign_fields = [
    'name',
    'objective',
    'stop_time',
    'start_time',
    'status',
    'created_time'
]

campaign_insight_fields = [
    'spend',
    'clicks',
    'cpc',
    'inline_link_clicks',
    'cost_per_inline_link_click',
    'impressions',
    'reach',
    'conversions',
    'cost_per_conversion',
    'inline_post_engagement',
    'cost_per_inline_post_engagement'
]

In [ ]:
campaign_list = []
i = 0

for campaign in campaigns:
    
    # Retrieving data from API
    attributes = campaign.api_get(fields=campaign_fields)
    insights = campaign.get_insights(fields=campaign_insight_fields)
    
    
    # Attributes
    name = None
    if 'name' in attributes and attributes['name']:
        name = attributes['name']
    
    objective = None
    if 'objective' in attributes and attributes['objective']:
        objective = attributes['objective']
    
    created_time = None
    if 'created_time' in attributes and attributes['created_time']:
        created_time = datetime.strptime(attributes['created_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')

    start_time = None
    if 'start_time' in attributes and attributes['start_time']:
        start_time = datetime.strptime(attributes['start_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')

    stop_time = None
    if 'stop_time' in attributes and attributes['stop_time']:
        stop_time = datetime.strptime(attributes['stop_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')
      
    status = None
    if 'status' in attributes and attributes['status']:
        status = attributes['status']
    
    
    # Insights
    for insight in insights:
        
        spend = None
        if 'spend' in insight and insight['spend']:
            spend = float(insight['spend'])

        clicks = None
        if 'spend' in insight and insight['clicks']:
            clicks = int(insight['clicks'])

        cpc = None
        if 'cpc' in insight and insight['cpc']:
            cpc = float(insight['cpc'])

        inline_link_clicks = None
        if 'inline_link_clicks' in insight and insight['inline_link_clicks']:
            inline_link_clicks = int(insight['inline_link_clicks'])

        cost_per_inline_link_click = None
        if 'cost_per_inline_link_click' in insight and insight['cost_per_inline_link_click']:
            cost_per_inline_link_click = float(insight['cost_per_inline_link_click'])

        impressions = None
        if 'impressions' in insight and insight['impressions']:
            impressions = int(insight['impressions'])

        reach = None
        if 'reach' in insight and insight['reach']:
            reach = int(insight['reach'])
        
        conversions = None
        if 'conversions' in insight and insight['conversions']:
            if insight['conversions']:
                for conversion in insight['conversions']:
                    conversion['value'] = int(conversion['value'])
            conversions = str(insight['conversions']).replace("'", '"')

        cost_per_conversion = None
        if 'cost_per_conversion' in insight and insight['cost_per_conversion']:
            if insight['cost_per_conversion']:
                for cost in insight['cost_per_conversion']:
                    cost['value'] = float(cost['value'])
            cost_per_conversion = str(insight['cost_per_conversion']).replace("'", '"')

        inline_post_engagement = None
        if 'inline_post_engagement' in insight and insight['inline_post_engagement']:
            inline_post_engagement = int(insight['inline_post_engagement'])

        cost_per_inline_post_engagement = None
        if 'cost_per_inline_post_engagement' in insight and insight['cost_per_inline_post_engagement']:
            cost_per_inline_post_engagement = float(insight['cost_per_inline_post_engagement'])
    
    campaign_temp_dict = {'id':campaign['id'],
                'name':name,
                'objective':objective,
                'created_time':created_time,  
                'start_time':start_time,          
                'stop_time':stop_time, 
                'status':status,
                'spend':spend,
                'clicks':clicks,
                'cpc':cpc,
                'inline_link_clicks':inline_link_clicks,
                'cost_per_inline_link_click':cost_per_inline_link_click,
                'impressions':impressions,
                'reach':reach,
                'conversions':conversions,
                'cost_per_conversion':cost_per_conversion,
                'inline_post_engagement':inline_post_engagement,
                'cost_per_inline_post_engagement':cost_per_inline_post_engagement}
    
    campaign_list.append(campaign_temp_dict)
    
    i = i +1
    if i%10 == 0:
        print("Requisição: ", i)
    
    time.sleep(10)

In [ ]:
pg_driver.run('DROP TABLE IF EXISTS lext.fb_ads_campaigns')

pg_driver.run('''create Table if not exists lext.fb_ads_campaigns (
id TEXT PRIMARY KEY,
name TEXT,
objective TEXT,
created_time TIMESTAMP,
start_time TIMESTAMP,
stop_time TIMESTAMP,
status TEXT,
spend FLOAT,
clicks BIGINT,
cpc FLOAT,
inline_link_clicks INT,
cost_per_inline_link_click FLOAT,
impressions INT,
reach INT,
conversions TEXT,
cost_per_conversion TEXT,
inline_post_engagement INT, 
cost_per_inline_post_engagement FLOAT);''')

pg_driver.run('''DELETE FROM lext.fb_ads_campaigns''')

for campaing in campaign_list:
    pg_driver.run('''INSERT INTO lext.fb_ads_campaigns VALUES (
    %(id)s,
    %(name)s,
    %(objective)s,
    %(created_time)s,
    %(start_time)s,
    %(stop_time)s,
    %(status)s,
    %(spend)s,
    %(clicks)s,
    %(cpc)s,
    %(inline_link_clicks)s,
    %(cost_per_inline_link_click)s,
    %(impressions)s,
    %(reach)s,
    %(conversions)s,
    %(cost_per_conversion)s,
    %(inline_post_engagement)s,
    %(cost_per_inline_post_engagement)s
    )''', campaing)
    
pg_driver.commit()

In [ ]:
# Get adsets's ids
adsets = my_account.get_ad_sets()

In [ ]:
# Adset's fields and params
adset_fields = [
    'campaign_id',
    'name',
    'billing_event',
    'optimization_goal',
    'created_time',
    'start_time',
    'end_time',
    'status',
    'targeting'
]

adset_insight_fields = [
    'spend',
    'clicks',
    'cpc',
    'inline_link_clicks',
    'cost_per_inline_link_click',
    'impressions',
    'reach',
    'conversions',
    'cost_per_conversion',
    'inline_post_engagement',
    'cost_per_inline_post_engagement'
]

In [ ]:
adsets_list = []
i = 0

for adset in adsets:
    
    # Retrieving data from API
    attributes = adset.api_get(fields=adset_fields)
    insights = adset.get_insights(fields=adset_insight_fields)
    
    
    # Attributes
    campaign_id = None
    if 'campaign_id' in attributes and attributes['campaign_id']:
        campaign_id = attributes['campaign_id']
    
    name = None
    if 'name' in attributes and attributes['name']:
        name = attributes['name']
    
    billing_event = None
    if 'billing_event' in attributes and attributes['billing_event']:
        billing_event = attributes['billing_event']
        
    optimization_goal = None
    if 'optimization_goal' in attributes and attributes['optimization_goal']:
        optimization_goal = attributes['optimization_goal']
        
    created_time = None
    if 'created_time' in attributes and attributes['created_time']:
        created_time = datetime.strptime(attributes['created_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')
    
    start_time = None
    if 'start_time' in attributes and attributes['start_time']:
        start_time = datetime.strptime(attributes['start_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')
    
    end_time = None
    if 'end_time' in attributes and attributes['end_time']:
        end_time = datetime.strptime(attributes['end_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')
    
    status = None
    if 'status' in attributes and attributes['status']:
        status = attributes['status']
        
    publisher_platforms = None
    if 'targeting' in attributes and attributes['targeting']:
        if 'publisher_platforms' in attributes['targeting'] and attributes['targeting']['publisher_platforms']:
            publisher_platforms = ', '.join([str(platform) for platform in attributes['targeting']['publisher_platforms']])
            
    
    # Insights
    for insight in insights:
        
        spend = None
        if 'spend' in insight and insight['spend']:
            spend = float(insight['spend'])

        clicks = None
        if 'spend' in insight and insight['clicks']:
            clicks = int(insight['clicks'])

        cpc = None
        if 'cpc' in insight and insight['cpc']:
            cpc = float(insight['cpc'])

        inline_link_clicks = None
        if 'inline_link_clicks' in insight and insight['inline_link_clicks']:
            inline_link_clicks = int(insight['inline_link_clicks'])

        cost_per_inline_link_click = None
        if 'cost_per_inline_link_click' in insight and insight['cost_per_inline_link_click']:
            cost_per_inline_link_click = float(insight['cost_per_inline_link_click'])

        impressions = None
        if 'impressions' in insight and insight['impressions']:
            impressions = int(insight['impressions'])

        reach = None
        if 'reach' in insight and insight['reach']:
            reach = int(insight['reach'])
        
        conversions = None
        if 'conversions' in insight and insight['conversions']:
            if insight['conversions']:
                for conversion in insight['conversions']:
                    conversion['value'] = int(conversion['value'])
            conversions = str(insight['conversions']).replace("'", '"')

        cost_per_conversion = None
        if 'cost_per_conversion' in insight and insight['cost_per_conversion']:
            if insight['cost_per_conversion']:
                for cost in insight['cost_per_conversion']:
                    cost['value'] = float(cost['value'])
            cost_per_conversion = str(insight['cost_per_conversion']).replace("'", '"')

        inline_post_engagement = None
        if 'inline_post_engagement' in insight and insight['inline_post_engagement']:
            inline_post_engagement = int(insight['inline_post_engagement'])

        cost_per_inline_post_engagement = None
        if 'cost_per_inline_post_engagement' in insight and insight['cost_per_inline_post_engagement']:
            cost_per_inline_post_engagement = float(insight['cost_per_inline_post_engagement'])
    
    adset_temp_dict = {'id':adset['id'],
                'campaign_id':campaign_id,
                'name':name,
                'billing_event':billing_event,
                'optimization_goal':optimization_goal,
                'created_time':created_time,
                'start_time':start_time,
                'end_time':end_time, 
                'status':status,
                'publisher_platforms':publisher_platforms,
                'spend':spend,
                'clicks':clicks,
                'cpc':cpc,
                'inline_link_clicks':inline_link_clicks,
                'cost_per_inline_link_click':cost_per_inline_link_click,
                'impressions':impressions,
                'reach':reach,
                'conversions':conversions,
                'cost_per_conversion':cost_per_conversion,
                'inline_post_engagement':inline_post_engagement,
                'cost_per_inline_post_engagement':cost_per_inline_post_engagement}
    
    adsets_list.append(adset_temp_dict)
    
    i = i +1
    if i%10 == 0:
        print("Requisição: ", i)
    
    time.sleep(10)

In [ ]:
adsets_list

In [ ]:
pg_driver.run('DROP TABLE IF EXISTS lext.fb_ads_adsets')

pg_driver.run('''create Table if not exists lext.fb_ads_adsets (
id TEXT PRIMARY KEY,
campaign_id TEXT,
name TEXT,
billing_event TEXT,
optimization_goal TEXT,
created_time TIMESTAMP,
start_time TIMESTAMP,
end_time TIMESTAMP,
status TEXT,
publisher_platforms TEXT,
spend FLOAT,
clicks BIGINT,
cpc FLOAT,
inline_link_clicks INT,
cost_per_inline_link_click FLOAT,
impressions INT,
reach INT,
conversions TEXT,
cost_per_conversion TEXT,
inline_post_engagement INT, 
cost_per_inline_post_engagement FLOAT);''')

pg_driver.run('''DELETE FROM lext.fb_ads_adsets''')

for adset in adsets_list:
    pg_driver.run('''INSERT INTO lext.fb_ads_adsets VALUES (
    %(id)s,
    %(campaign_id)s,
    %(name)s,
    %(billing_event)s,
    %(optimization_goal)s,
    %(created_time)s,
    %(start_time)s,
    %(end_time)s, 
    %(status)s,
    %(publisher_platforms)s,
    %(spend)s,
    %(clicks)s,
    %(cpc)s,
    %(inline_link_clicks)s,
    %(cost_per_inline_link_click)s,
    %(impressions)s,
    %(reach)s,
    %(conversions)s,
    %(cost_per_conversion)s,
    %(inline_post_engagement)s,
    %(cost_per_inline_post_engagement)s
    )''', adset)
    
pg_driver.commit()

In [ ]:
# Get ads
ads = my_account.get_ads()

In [ ]:
# Adset's fields and params
ad_fields = [
    'campaign_id',
    'adset_id',
    'name',
    'created_time',
    'status'
]

ad_insight_fields = [
    'date_start',
    'date_stop',
    'spend',
    'clicks',
    'cpc',
    'inline_link_clicks',
    'cost_per_inline_link_click',
    'impressions',
    'reach',
    'conversions',
    'cost_per_conversion',
    'inline_post_engagement',
    'cost_per_inline_post_engagement'
]

creative_fields = [
    'image_url'
]

In [ ]:
ads_list = []
i = 0

for ad in ads:
    
    # Retrieving data from API
    attributes = ad.api_get(fields=ad_fields)
    insights = ad.get_insights(fields=ad_insight_fields)
    
    # Attributes
    campaign_id = None
    if 'campaign_id' in attributes and attributes['campaign_id']:
        campaign_id = attributes['campaign_id']
        
    adset_id = None
    if 'adset_id' in attributes and attributes['adset_id']:
        adset_id = attributes['adset_id']    
    
    name = None
    if 'name' in attributes and attributes['name']:
        name = attributes['name']
        
    created_time = None
    if 'created_time' in attributes and attributes['created_time']:
        created_time = datetime.strptime(attributes['created_time'], "%Y-%m-%dT%H:%M:%S-0300").strftime('%m/%d/%Y, %H:%M:%S')
    
    status = None
    if 'status' in attributes and attributes['status']:
        status = attributes['status']
    
    # Get Ad's Creatives
    creatives = ad.get_ad_creatives()
    for creative in creatives:
        creative_api = creative.api_get(fields=creative_fields)
        
        image_url = None
        if "image_url" in creative_api and creative_api["image_url"]:
            image_url = creative_api["image_url"]
    
    date_start = None
    date_stop = None
    spend = None
    clicks = None
    cpc = None
    inline_link_clicks = None
    cost_per_inline_link_click = None
    impressions = None
    reach = None
    conversions = None
    cost_per_conversion = None
    inline_post_engagement = None
    cost_per_inline_post_engagement = None 
    
    # Insights
    for insight in insights:
        
        if 'date_start' in insight and insight['date_start']:
            date_start = datetime.strptime(insight['date_start'], "%Y-%m-%d").strftime('%m/%d/%Y')
    
        if 'date_stop' in insight and insight['date_stop']:
            date_stop = datetime.strptime(insight['date_stop'], "%Y-%m-%d").strftime('%m/%d/%Y')
        
        if 'spend' in insight and insight['spend']:
            spend = float(insight['spend'])

        if 'spend' in insight and insight['clicks']:
            clicks = int(insight['clicks'])

        if 'cpc' in insight and insight['cpc']:
            cpc = float(insight['cpc'])

        if 'inline_link_clicks' in insight and insight['inline_link_clicks']:
            inline_link_clicks = int(insight['inline_link_clicks'])

        if 'cost_per_inline_link_click' in insight and insight['cost_per_inline_link_click']:
            cost_per_inline_link_click = float(insight['cost_per_inline_link_click'])

        if 'impressions' in insight and insight['impressions']:
            impressions = int(insight['impressions'])

        if 'reach' in insight and insight['reach']:
            reach = int(insight['reach'])

        if 'conversions' in insight and insight['conversions']:
            if insight['conversions']:
                for conversion in insight['conversions']:
                    conversion['value'] = int(conversion['value'])
            conversions = str(insight['conversions']).replace("'", '"')

        if 'cost_per_conversion' in insight and insight['cost_per_conversion']:
            if insight['cost_per_conversion']:
                for cost in insight['cost_per_conversion']:
                    cost['value'] = float(cost['value'])
            cost_per_conversion = str(insight['cost_per_conversion']).replace("'", '"')

        if 'inline_post_engagement' in insight and insight['inline_post_engagement']:
            inline_post_engagement = int(insight['inline_post_engagement'])

        if 'cost_per_inline_post_engagement' in insight and insight['cost_per_inline_post_engagement']:
            cost_per_inline_post_engagement = float(insight['cost_per_inline_post_engagement'])
    
    ad_temp_dict = {'id':ad['id'],
                'campaign_id':campaign_id,
                'adset_id':adset_id,
                'name':name,
                'created_time':created_time,
                'status':status,
                'image_url':image_url,
                'date_start':date_start,
                'date_stop':date_stop, 
                'spend':spend,
                'clicks':clicks,
                'cpc':cpc,
                'inline_link_clicks':inline_link_clicks,
                'cost_per_inline_link_click':cost_per_inline_link_click,
                'impressions':impressions,
                'reach':reach,
                'conversions':conversions,
                'cost_per_conversion':cost_per_conversion,
                'inline_post_engagement':inline_post_engagement,
                'cost_per_inline_post_engagement':cost_per_inline_post_engagement}
    ads_list.append(ad_temp_dict)
    
    i = i +1
    if i%50 == 0:
        print("Requisição: ", i)
    
    time.sleep(20)

In [ ]:
pg_driver.run('DROP TABLE IF EXISTS lext.fb_ads_ads')

pg_driver.run('''create Table if not exists lext.fb_ads_ads (
id TEXT PRIMARY KEY,
campaign_id TEXT,
adset_id TEXT,
name TEXT,
created_time TIMESTAMP,
status TEXT,
image_url TEXT,
date_start TIMESTAMP,
date_stop TIMESTAMP,
spend FLOAT,
clicks BIGINT,
cpc FLOAT,
inline_link_clicks INT,
cost_per_inline_link_click FLOAT,
impressions INT,
reach INT,
conversions TEXT,
cost_per_conversion TEXT,
inline_post_engagement INT, 
cost_per_inline_post_engagement FLOAT);''')

pg_driver.run('''DELETE FROM lext.fb_ads_ads''')

for ad in ads_list:
    pg_driver.run('''INSERT INTO lext.fb_ads_ads VALUES (
    %(id)s,
    %(campaign_id)s,
    %(adset_id)s,
    %(name)s,
    %(created_time)s,
    %(status)s,
    %(image_url)s,
    %(date_start)s,
    %(date_stop)s, 
    %(spend)s,
    %(clicks)s,
    %(cpc)s,
    %(inline_link_clicks)s,
    %(cost_per_inline_link_click)s,
    %(impressions)s,
    %(reach)s,
    %(conversions)s,
    %(cost_per_conversion)s,
    %(inline_post_engagement)s,
    %(cost_per_inline_post_engagement)s
    )''', ad)
    
pg_driver.commit()

In [ ]:
#############
#ANALYZE ADS#
#############
tables = ['fb_ads_campaigns', 'fb_ads_adsets', 'fb_ads_ads']

conn = pg_driver._get_connection()
old_isolation_level = conn.isolation_level
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)

for table in tables:
    print(F'{datetime.utcnow()} analyzing lext.{table}')
    pg_driver.run(f"VACUUM ANALYZE lext.{table}")

conn.set_isolation_level(old_isolation_level)
pg_driver._unset_connection()
print('Done')